In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'c'
project = 'Intermediate SQL'
data = make_dir_line(modality, project)

raw = data('raw')

In [4]:
import sqlite3

conn = sqlite3.connect(":memory:")  ## aca se indica el nombre de la db.
cur = conn.cursor()

# 3.4.0 Sorting and Grouping

In [5]:
conn.executescript(
    """
DROP TABLE IF EXISTS films;

CREATE TABLE films (
    id            INT,
    title         STRING,
    release_year  INT,
    country       STRING,
    duration      INT,
    language      STRING,
    certification STRING,
    gross         DOUBLE,
    budget        DOUBLE
);

DROP TABLE IF EXISTS people;

CREATE TABLE people (
    id        INT,
    name      STRING,
    birthdate STRING,
    deathdate STRING
);

DROP TABLE IF EXISTS reviews;

CREATE TABLE reviews (
    id             INT,
    film_id        INT,
    num_user       INT,
    num_critic     INT,
    imdb_score     DOUBLE,
    num_votes      DOUBLE,
    facebook_likes DOUBLE
);

DROP TABLE IF EXISTS roles;

CREATE TABLE roles (
    id        INT,
    film_id   INT,
    person_id INT,
    role      STRING
);

"""
)
conn.commit()

In [6]:
df = pd.read_csv(raw / 'films.csv', sep = ',', decimal = '.', header = None, encoding = 'utf-8')
df.columns = ['id','title','release_year','country','duration','language','certification','gross','budget']
films = list(zip(df.id, df.title, df.release_year, df.country, df.duration, df.language, df.certification, df.gross, df.budget))
cur.executemany("INSERT INTO films VALUES  (?,?,?,?,?,?,?,?,?)", films)

In [7]:
df = pd.read_csv(raw / 'people.csv', sep = ',', decimal = '.', header = None, encoding = 'utf-8')
df.columns = ['id','name','birthdate','deathdate']
people = list(zip(df.id, df.name, df.birthdate, df.deathdate))
cur.executemany("INSERT INTO people VALUES  (?,?,?,?)", people)

In [8]:
df = pd.read_csv(raw / 'reviews.csv', sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
df.columns = ['id','film_id','num_user','num_critic','imdb_score','num_votes','facebook_likes']
df = df.loc[:,['id','film_id','num_user','num_critic','imdb_score','num_votes','facebook_likes']]
reviews = list(zip(df.id, df.film_id, df.num_user, df.num_critic, df.imdb_score, df.num_votes, df.facebook_likes))
cur.executemany("INSERT INTO reviews VALUES  (?,?,?,?,?,?,?)", reviews)

In [9]:
df = pd.read_csv(raw / 'roles.csv', sep = ',', decimal = '.', header = None, encoding = 'utf-8')
df.columns = ['id','film_id','person_id','role']
roles = list(zip(df.id, df.film_id, df.person_id, df.role))
cur.executemany("INSERT INTO roles VALUES  (?,?,?,?)", roles)

## 3.4.2 Sorting single fields

In [10]:
# -- Select name from people and sort alphabetically
cur.execute("SELECT name FROM people ORDER BY name;").fetchall()

[('50 Cent',),
 ('A. Michael Baldwin',),
 ('A. Raven Cruz',),
 ('A.J. Buckley',),
 ('A.J. DeLucia',),
 ('A.J. Langer',),
 ('AJ Michalka',),
 ('Aaliyah',),
 ('Aaron Ashmore',),
 ('Aaron Hann',),
 ('Aaron Hill',),
 ('Aaron Hughes',),
 ('Aaron Kwok',),
 ('Aaron Schneider',),
 ('Aaron Seltzer',),
 ('Aaron Stanford',),
 ('Aaron Staton',),
 ('Aaron Yoo',),
 ('Aasheekaa Bathija',),
 ('Aasif Mandvi',),
 ('Abbie Cornish',),
 ('Abby Elliott',),
 ('Abby Mukiibi Nkaaga',),
 ('Abel Ferrara',),
 ('Abhishek Bachchan',),
 ('Abigail Evans',),
 ('Abigail Spencer',),
 ('Abraham Benrubi',),
 ('Ace Marrero',),
 ('Adam Alexi-Malle',),
 ('Adam Arkin',),
 ('Adam Baldwin',),
 ('Adam Boyer',),
 ('Adam Brooks',),
 ('Adam Brown',),
 ('Adam Butcher',),
 ('Adam Carolla',),
 ('Adam Copeland',),
 ('Adam DiMarco',),
 ('Adam Garcia',),
 ('Adam Goldberg',),
 ('Adam Green',),
 ('Adam Hicks',),
 ('Adam Jay Epstein',),
 ('Adam Lamberg',),
 ('Adam LeFevre',),
 ('Adam Marcus',),
 ('Adam McKay',),
 ('Adam Rapp',),
 ('Adam Rat

In [11]:
# -- Select name from people and sort alphabetically
cur.execute("Select title, duration FROM films ORDER BY duration DESC;").fetchall()

[('Carlos', 334),
 ('Blood In, Blood Out', 330),
 ("Heaven's Gate", 325),
 ('The Legend of Suriyothai', 300),
 ('Das Boot', 293),
 ('Apocalypse Now', 289),
 ('The Company', 286),
 ('Gods and Generals', 280),
 ('Gettysburg', 271),
 ('Arn: The Knight Templar', 270),
 ('Cleopatra', 251),
 ('Once Upon a Time in America', 251),
 ('Gandhi', 240),
 ('The Wolf of Wall Street', 240),
 ('Emma', 240),
 ('Dances with Wolves', 236),
 ('Lawrence of Arabia', 227),
 ('Gone with the Wind', 226),
 ('The Greatest Story Ever Told', 225),
 ('The Godfather: Part II', 220),
 ('All the Pretty Horses', 220),
 ('The Last Emperor', 219),
 ('Gangs of New York', 216),
 ('Woodstock', 215),
 ('The Thin Red Line', 215),
 ('Watchmen', 215),
 ('Wyatt Earp', 212),
 ('Nixon', 212),
 ('JFK', 206),
 ('Alexander', 206),
 ('Seven Samurai', 202),
 ('Malcolm X', 202),
 ('King Kong', 201),
 ('King Kong', 201),
 ('King Kong', 201),
 ('Doctor Zhivago', 200),
 ("It's a Mad, Mad, Mad, Mad World", 197),
 ('Deadline Gallipoli', 197),

## 3.4.3 Sorting multiple fields

In [12]:
# -- Select name from people and sort alphabetically
cur.execute("SELECT release_year, duration, title FROM films ORDER BY release_year DESC, duration DESC;").fetchall()

[(2016, 183, 'Batman v Superman: Dawn of Justice'),
 (2016, 156, 'The Wailing'),
 (2016, 147, 'Captain America: Civil War'),
 (2016, 144, '13 Hours'),
 (2016, 144, 'X-Men: Apocalypse'),
 (2016, 141, 'Ben-Hur'),
 (2016, 141, 'Ben-Hur'),
 (2016, 141, 'Ben-Hur'),
 (2016, 139, 'Free State of Jones'),
 (2016, 134, 'Race'),
 (2016, 134, 'The Conjuring 2'),
 (2016, 130, 'Airlift'),
 (2016, 129, 'Now You See Me 2'),
 (2016, 127, 'The Infiltrator'),
 (2016, 126, 'Gods of Egypt'),
 (2016, 123, 'Jason Bourne'),
 (2016, 123, 'Suicide Squad'),
 (2016, 123, 'Warcraft'),
 (2016, 122, 'Star Trek Beyond'),
 (2016, 120, 'Allegiant'),
 (2016, 120, "God's Not Dead 2"),
 (2016, 120, 'Godzilla Resurgence'),
 (2016, 120, 'Godzilla Resurgence'),
 (2016, 120, 'Independence Day: Resurgence'),
 (2016, 120, 'The Birth of a Nation'),
 (2016, 120, "The Huntsman: Winter's War"),
 (2016, 119, 'Xi you ji zhi: Sun Wukong san da Baigu Jing'),
 (2016, 118, 'The Neon Demon'),
 (2016, 117, 'The BFG'),
 (2016, 117, 'The Fin

In [13]:
# -- Select the certification, release year, and title sorted by certification and release year
cur.execute("SELECT certification, release_year, title FROM films ORDER BY certification ASC, release_year DESC;").fetchall()

[(None, 2016, "A Beginner's Guide to Snuff"),
 (None, 2016, 'Airlift'),
 (None, 2016, "Alleluia! The Devil's Carnival"),
 (None, 2016, 'Antibirth'),
 (None, 2016, 'Fight Valley'),
 (None, 2016, 'Godzilla Resurgence'),
 (None, 2016, 'Godzilla Resurgence'),
 (None, 2016, 'Irreplaceable'),
 (None, 2016, 'Kickboxer: Vengeance'),
 (None, 2016, 'Operation Chromite'),
 (None, 2016, 'Restoration'),
 (None, 2016, 'The Little Ponderosa Zoo'),
 (None, 2016, 'Two Lovers and a Bear'),
 (None, 2016, 'Xi you ji zhi: Sun Wukong san da Baigu Jing'),
 (None, 2015, 'A Tale of Three Cities'),
 (None, 2015, "A Warrior's Tail"),
 (None, 2015, 'Abandoned'),
 (None, 2015, 'Adulterers'),
 (None, 2015, 'America Is Still the Place'),
 (None, 2015, 'American Hero'),
 (None, 2015, "Animal Kingdom: Let's go Ape"),
 (None, 2015, 'Antarctic Edge: 70Â° South'),
 (None, 2015, 'Baahubali: The Beginning'),
 (None, 2015, 'Bleeding Hearts'),
 (None, 2015, 'Censored Voices'),
 (None, 2015, 'Checkmate'),
 (None, 2015, 'Chiam

## 3.4.5 GROUP BY single fields

In [14]:
# -- Find the release_year and film_count of each year
cur.execute("SELECT release_year, COUNT(*) AS film_count FROM films GROUP BY release_year ORDER BY release_year;").fetchall()

[(None, 42),
 (1916, 1),
 (1920, 1),
 (1925, 1),
 (1927, 1),
 (1929, 2),
 (1930, 1),
 (1932, 1),
 (1933, 2),
 (1934, 1),
 (1935, 1),
 (1936, 2),
 (1937, 2),
 (1938, 2),
 (1939, 3),
 (1940, 5),
 (1941, 1),
 (1942, 2),
 (1943, 1),
 (1944, 1),
 (1945, 4),
 (1946, 3),
 (1947, 3),
 (1948, 3),
 (1949, 2),
 (1950, 1),
 (1951, 3),
 (1952, 4),
 (1953, 4),
 (1954, 5),
 (1955, 2),
 (1956, 3),
 (1957, 2),
 (1958, 1),
 (1959, 3),
 (1960, 3),
 (1961, 5),
 (1962, 8),
 (1963, 8),
 (1964, 10),
 (1965, 7),
 (1966, 6),
 (1967, 4),
 (1968, 11),
 (1969, 10),
 (1970, 12),
 (1971, 11),
 (1972, 9),
 (1973, 9),
 (1974, 9),
 (1975, 6),
 (1976, 10),
 (1977, 16),
 (1978, 16),
 (1979, 16),
 (1980, 24),
 (1981, 33),
 (1982, 30),
 (1983, 22),
 (1984, 31),
 (1985, 29),
 (1986, 26),
 (1987, 32),
 (1988, 31),
 (1989, 33),
 (1990, 30),
 (1991, 31),
 (1992, 34),
 (1993, 48),
 (1994, 54),
 (1995, 70),
 (1996, 99),
 (1997, 118),
 (1998, 134),
 (1999, 168),
 (2000, 171),
 (2001, 188),
 (2002, 209),
 (2003, 169),
 (2004, 214

In [15]:
# -- Find the release_year and average duration of films for each year
cur.execute("SELECT release_year, AVG(duration) AS avg_duration FROM films GROUP BY release_year ORDER BY release_year;").fetchall()

[(None, 77.4390243902439),
 (1916, 123.0),
 (1920, 110.0),
 (1925, 151.0),
 (1927, 145.0),
 (1929, 105.0),
 (1930, 96.0),
 (1932, 79.0),
 (1933, 77.5),
 (1934, 65.0),
 (1935, 81.0),
 (1936, 93.5),
 (1937, 92.0),
 (1938, 116.0),
 (1939, 149.33333333333334),
 (1940, 108.0),
 (1941, 118.0),
 (1942, 76.0),
 (1943, 122.0),
 (1944, 101.0),
 (1945, 103.75),
 (1946, 144.66666666666666),
 (1947, 101.66666666666667),
 (1948, 98.66666666666667),
 (1949, 106.0),
 (1950, 107.0),
 (1951, 134.66666666666666),
 (1952, 106.75),
 (1953, 106.25),
 (1954, 140.6),
 (1955, 112.5),
 (1956, 103.0),
 (1957, 128.5),
 (1958, 108.0),
 (1959, 136.66666666666666),
 (1960, 126.33333333333333),
 (1961, 145.6),
 (1962, 141.375),
 (1963, 153.875),
 (1964, 119.4),
 (1965, 155.0),
 (1966, 119.16666666666667),
 (1967, 105.75),
 (1968, 120.54545454545455),
 (1969, 126.0),
 (1970, 133.16666666666666),
 (1971, 105.54545454545455),
 (1972, 115.33333333333333),
 (1973, 111.33333333333333),
 (1974, 113.77777777777777),
 (1975, 

## 3.4.6 GROUP BY multiple fields

In [16]:
# -- Find the release_year, country, and max_budget, then group and order by release_year and country
cur.execute("SELECT release_year, country, MAX(budget) AS max_budget FROM films GROUP BY release_year, country ORDER BY release_year, country;").fetchall()

[(None, None, None),
 (None, 'Australia', 15000000.0),
 (None, 'Canada', None),
 (None, 'France', None),
 (None, 'Iceland', None),
 (None, 'Japan', None),
 (None, 'Poland', None),
 (None, 'Sweden', None),
 (None, 'UK', None),
 (None, 'USA', 5000000.0),
 (1916, 'USA', 385907.0),
 (1920, 'USA', 100000.0),
 (1925, 'USA', 245000.0),
 (1927, 'Germany', 6000000.0),
 (1929, 'Germany', None),
 (1929, 'USA', 379000.0),
 (1930, 'USA', 3950000.0),
 (1932, 'USA', 800000.0),
 (1933, 'USA', 439000.0),
 (1934, 'USA', 325000.0),
 (1935, 'USA', 609000.0),
 (1936, 'USA', 1500000.0),
 (1937, 'USA', 2000000.0),
 (1938, 'USA', 2000000.0),
 (1939, 'USA', 3977000.0),
 (1940, 'USA', 2600000.0),
 (1941, 'USA', 1250000.0),
 (1942, 'USA', 950000.0),
 (1943, 'USA', 2627000.0),
 (1944, 'USA', 2361000.0),
 (1945, 'USA', 2160000.0),
 (1946, 'USA', 8000000.0),
 (1947, 'USA', 3209000.0),
 (1948, 'USA', 3700000.0),
 (1949, 'USA', 1600000.0),
 (1950, 'USA', 3768785.0),
 (1951, 'USA', 7623000.0),
 (1952, 'USA', 4000000.0

## 3.4.7 Answering business questions

Which release_year had the most language diversity?

In [17]:
cur.execute("SELECT release_year, COUNT(DISTINCT language) AS language FROM films GROUP BY release_year ORDER BY language DESC;").fetchall()

[(2006, 16),
 (2015, 15),
 (2005, 14),
 (2013, 13),
 (2008, 13),
 (2009, 12),
 (2004, 12),
 (2007, 11),
 (2010, 10),
 (2014, 9),
 (2011, 9),
 (2012, 8),
 (2003, 8),
 (2001, 8),
 (2000, 8),
 (2002, 7),
 (1998, 7),
 (2016, 6),
 (1997, 5),
 (None, 5),
 (1996, 4),
 (1999, 3),
 (1995, 3),
 (1993, 3),
 (1981, 3),
 (1972, 3),
 (1964, 3),
 (1994, 2),
 (1992, 2),
 (1988, 2),
 (1985, 2),
 (1983, 2),
 (1970, 2),
 (1969, 2),
 (1967, 2),
 (1966, 2),
 (1965, 2),
 (1955, 2),
 (1954, 2),
 (1929, 2),
 (1991, 1),
 (1990, 1),
 (1989, 1),
 (1987, 1),
 (1986, 1),
 (1984, 1),
 (1982, 1),
 (1980, 1),
 (1979, 1),
 (1978, 1),
 (1977, 1),
 (1976, 1),
 (1975, 1),
 (1974, 1),
 (1973, 1),
 (1971, 1),
 (1968, 1),
 (1963, 1),
 (1962, 1),
 (1961, 1),
 (1960, 1),
 (1959, 1),
 (1958, 1),
 (1957, 1),
 (1956, 1),
 (1953, 1),
 (1952, 1),
 (1951, 1),
 (1950, 1),
 (1949, 1),
 (1948, 1),
 (1947, 1),
 (1946, 1),
 (1945, 1),
 (1944, 1),
 (1943, 1),
 (1942, 1),
 (1941, 1),
 (1940, 1),
 (1939, 1),
 (1938, 1),
 (1937, 1),
 (1936,

## 3.4.9 Filter with HAVING

In [18]:
# -- Select the country and distinct count of certification as certification_count, Group by country, Filter results to countries with more than 10 different certifications
cur.execute("SELECT country, COUNT(DISTINCT certification) AS certification_count FROM films GROUP BY country HAVING COUNT(DISTINCT certification) > 10;").fetchall()

[('USA', 12)]

## 3.4.10 HAVING and sorting

In [19]:
# -- Select the country and average_budget from films, Group by country, Filter to countries with an average_budget of more than one billion, Order by descending order of the aggregated budget
cur.execute("SELECT country, ROUND(AVG(budget),2) AS average_budget FROM films GROUP BY country HAVING ROUND(AVG(budget),2) > 1000000000 ORDER BY average_budget DESC;").fetchall()

[('South Korea', 1383960000.0), ('Hungary', 1260000000.0)]

## 3.4.11 All together now

In [20]:
# -- -- Select the release_year for films released after 1990 grouped by year, Group by country, Filter to countries with an average_budget of more than one billion, Order by descending order of the aggregated budget
cur.execute("SELECT release_year FROM films GROUP BY release_year HAVING release_year > 1990 ORDER BY release_year ASC;").fetchall()

[(1991,),
 (1992,),
 (1993,),
 (1994,),
 (1995,),
 (1996,),
 (1997,),
 (1998,),
 (1999,),
 (2000,),
 (2001,),
 (2002,),
 (2003,),
 (2004,),
 (2005,),
 (2006,),
 (2007,),
 (2008,),
 (2009,),
 (2010,),
 (2011,),
 (2012,),
 (2013,),
 (2014,),
 (2015,),
 (2016,)]

In [21]:
# -- Modify the query to also list the average budget and average gross
cur.execute("SELECT release_year, AVG(budget) AS avg_budget, AVG(gross) AS avg_gross FROM films WHERE release_year > 1990 GROUP BY release_year;").fetchall()

[(1991, 25176548.387096774, 53844501.666666664),
 (1992, 25982030.303030305, 63665195.14705882),
 (1993, 20729787.23404255, 45302091.41304348),
 (1994, 29013773.58490566, 59395666.16981132),
 (1995, 32775000.0, 44909519.98550725),
 (1996, 31620612.244897958, 42044174.25263158),
 (1997, 59424490.74074074, 44793772.43103448),
 (1998, 40460000.0, 38377007.96124031),
 (1999, 38981780.487804875, 38072176.27710843),
 (2000, 34931375.75757576, 42172627.58083832),
 (2001, 37687307.692307696, 43255716.92553192),
 (2002, 32598510.68811881, 43511151.485),
 (2003, 37208648.833333336, 48727746.72327044),
 (2004, 46865344.62436548, 40726529.11),
 (2005, 70323938.23152709, 41159143.29064039),
 (2006, 93968929.57746479, 39237855.9537037),
 (2007, 35271131.14754099, 46267501.02234637),
 (2008, 41804885.5721393, 44573509.378109455),
 (2009, 37073287.03703704, 46207440.2),
 (2010, 46094664.948453605, 49908326.01005025),
 (2011, 37775251.71568628, 45785836.64397906),
 (2012, 41331819.162436545, 62873527.6

In [22]:
# -- Modify the query to see only years with an avg_budget of more than 60 million
cur.execute("SELECT release_year, AVG(budget) AS avg_budget, AVG(gross) AS avg_gross FROM films WHERE release_year > 1990 GROUP BY release_year HAVING AVG(budget) > 60000000;").fetchall()

[(2005, 70323938.23152709, 41159143.29064039),
 (2006, 93968929.57746479, 39237855.9537037)]

In [23]:
# -- Order the results from highest to lowest average gross and limit to one
cur.execute("SELECT release_year, AVG(budget) AS avg_budget, AVG(gross) AS avg_gross FROM films WHERE release_year > 1990 GROUP BY release_year HAVING AVG(budget) > 60000000 ORDER BY avg_gross DESC LIMIT 1;").fetchall()

[(2005, 70323938.23152709, 41159143.29064039)]

In [24]:
print('Ok_')

Ok_
